In [1]:
import telebot
from telebot import types
import sqlite3

In [2]:
bot = telebot.TeleBot(TOKEN)

In [3]:
conn = sqlite3.connect('guitars.db')
cursor = conn.cursor()

In [4]:
try:
    query = "CREATE TABLE \"guitars\" (\"ID\" INTEGER UNIQUE, \"user_id\" INTEGER, \"Request\" TEXT, \"Muztorg\" TEXT, \"PopMusic\" TEXT, \"Dynatone\" TEXT,  PRIMARY KEY (\"ID\"))"
    cursor.execute(query)
    
except:
    pass

In [5]:
@bot.message_handler(commands=['start'])
def send_keyboard_1(message, text="Привет, чем я могу тебе помочь?"):
    keyboard = types.ReplyKeyboardMarkup(row_width=3, resize_keyboard=True)
    itembtn1 = types.KeyboardButton('Бренды электрогитар')
    itembtn2 = types.KeyboardButton('Элементы конструкции')
    itembtn3 = types.KeyboardButton('Как заниматься?')
    itembtn4 = types.KeyboardButton('Немного о струнах')
    itembtn5 = types.KeyboardButton('Известные музыканты')
    itembtn6 = types.KeyboardButton('Что делать после покупки?')
    itembtn7 = types.KeyboardButton('Первая электрогитара')
    itembtn8 = types.KeyboardButton('Найти гитару в магазине')
    itembtn9 = types.KeyboardButton('Пока всё!')
    keyboard.add(itembtn1, itembtn2, itembtn3)
    keyboard.add(itembtn4, itembtn5, itembtn6)
    keyboard.add(itembtn7, itembtn8, itembtn9)

    msg = bot.send_message(message.from_user.id,
                     text=text, reply_markup=keyboard)

    bot.register_next_step_handler(msg, callback_worker)


In [6]:
def send_keyboard_2(message, text='Есть много различных брендов электрогитар, и у каждого есть свои интересные фишки. '
                    'Я могу рассказать о некоторых из них, которые полюбились в мире. '
                    'О каком бренде Вы хотели бы узнать поподробнее?'):
    keyboard = types.ReplyKeyboardMarkup(row_width=3, resize_keyboard=True)
    itembtn1 = types.KeyboardButton('Fender')
    itembtn2 = types.KeyboardButton('Gibson')
    itembtn3 = types.KeyboardButton('Epiphone')
    itembtn4 = types.KeyboardButton('Ibanez')
    itembtn5 = types.KeyboardButton('Yamaha')
    itembtn6 = types.KeyboardButton('Jackson')
    itembtn7 = types.KeyboardButton('Ashtone')
    itembtn8 = types.KeyboardButton('Вернуться назад')
    keyboard.add(itembtn1, itembtn2, itembtn3)
    keyboard.add(itembtn4, itembtn5, itembtn6) 
    keyboard.add(itembtn7, itembtn8)
    
    msg = bot.send_message(message.from_user.id,
                     text=text, reply_markup=keyboard) 

    bot.register_next_step_handler(msg, callback_worker)

In [7]:
def send_keyboard_3(message, text="О чем бы Вы хотели узнать более детально?"):
    keyboard = types.ReplyKeyboardMarkup(row_width=3, resize_keyboard=True)
    itembtn1 = types.KeyboardButton('Порода дерева')
    itembtn2 = types.KeyboardButton('Тип корпуса')
    itembtn3 = types.KeyboardButton('Гриф')
    itembtn4 = types.KeyboardButton('Звукосниматели')
    itembtn5 = types.KeyboardButton('Настройка тембра')
    itembtn6 = types.KeyboardButton('Рычаг тремоло')
    itembtn7 = types.KeyboardButton('Колки')
    itembtn8 = types.KeyboardButton('Бридж')
    itembtn9 = types.KeyboardButton('Вернуться назад')
    keyboard.add(itembtn1, itembtn2, itembtn3)
    keyboard.add(itembtn4, itembtn5, itembtn6) 
    keyboard.add(itembtn7, itembtn8, itembtn9)
    
    msg = bot.send_message(message.from_user.id,
                     text=text, reply_markup=keyboard)

    bot.register_next_step_handler(msg, callback_worker)

In [8]:
def send_keyboard_4(message, text="Вернуться назад?"):
    keyboard = types.ReplyKeyboardMarkup(row_width=2, resize_keyboard=True)
    itembtn1 = types.KeyboardButton('Вернуться назад')
    itembtn2 = types.KeyboardButton('История поиска')
    itembtn3 = types.KeyboardButton('Очистить истроию')
    itembtn4 = types.KeyboardButton('Удалить экземпляр поиска')
    keyboard.add(itembtn2, itembtn3)
    keyboard.add(itembtn4, itembtn1)
    
    msg = bot.send_message(message.from_user.id, 
                           text=text, reply_markup=keyboard)

    bot.register_next_step_handler(msg, callback_worker)

In [9]:
def search(message):
    msg = message.text
    url_muzt = 'https://www.muztorg.ru/search/' + msg.strip().lower().replace(' ', '%20')
    url_dyn = 'https://dynatone.ru/index.php?route=product/search&search=' + msg.strip().lower().replace(' ', '%20')
    url_pop = 'https://pop-music.ru/search/?q=' + msg.strip().lower().replace(' ', '+')
    bot.send_message(message.chat.id, 'Вот что мне удалось найти:\n\n' 
                    '<b>Музторг</b>\n'+ url_muzt + 
                    '\n\n<b>Dynatone</b>\n' + url_dyn +
                    '\n\n<b>Pop-Music</b>\n' + url_pop, parse_mode="html")
        
    with sqlite3.connect('guitars.db') as con:
        cursor = con.cursor()
        cursor.execute('INSERT INTO guitars (user_id, Request, Muztorg, PopMusic, Dynatone) VALUES (?, ?, ?, ?, ?)', 
                       (message.from_user.id, message.text, url_muzt, url_pop, url_dyn))
        con.commit()
        

In [10]:
def get_history(searches):
    searches_str = []
    try:
        for search in searches:
            searches_str.append('<b>' + str(search[0]) + ') ' + search[1] + ':</b>\n' + 
                                '<a href = "' + search[2] +'">'+'Музторг</a>'+ ', '+
                                '<a href = "' + search[3] +'">'+'Pop-Music</a>'+ ', '+
                                '<a href = "' + search[4] +'">'+'Dynatone</a>'+ '\n\n')
    except:
        searches_str = []
        
    return ''.join(searches_str)

def show_history(msg):
    with sqlite3.connect('guitars.db') as con:
        cursor = con.cursor()
        cursor.execute('SELECT ID, Request, Muztorg, PopMusic, Dynatone FROM guitars WHERE user_id=={}'.format(msg.from_user.id))
        searches = get_history(cursor.fetchall())
        
        if searches == '':
            bot.send_message(msg.chat.id, 'История поиска пуста')
        else:
            bot.send_message(msg.chat.id,'Вот, что Вы искали ранее:\n\n' + searches, parse_mode="html")
        

In [11]:
def delete_search(msg):
    numbers = msg.text.strip().split()
    for number in numbers:
        if number.isdigit():
            with sqlite3.connect('guitars.db') as con:
                cursor = con.cursor()
                cursor.execute('DELETE FROM guitars WHERE user_id==? AND ID==?', (msg.from_user.id, number))
                bot.send_message(msg.chat.id, 'Удалил из истории поиск №' + number)
        else:
            bot.send_message(msg.chat.id, 'Я могу удалять только по номеру поиска')

In [12]:
def delete_all_searches(msg):
    with sqlite3.connect('guitars.db') as con:
        cursor = con.cursor()
        cursor.execute('DELETE FROM guitars WHERE user_id=={}'.format(msg.from_user.id))
        con.commit()
    bot.send_message(msg.chat.id, 'История поиска очищена!')

In [13]:
def callback_worker(call):
    try:
        if call.text == 'Бренды электрогитар':
            send_keyboard_2(call)
        
        elif call.text == 'Элементы конструкции':
            send_keyboard_3(call)
        
        elif call.text == 'Вернуться назад':
            send_keyboard_1(call, 'Чем еще могу помочь?')
        
        elif call.text == 'Как заниматься?':
            bot.send_message(call.chat.id, 'Коротко о том, как должны проходить занятия на гитаре:\n\n'
                             '<b>1) По-немногу, но часто (желательно ежедневно по 1 часу)</b>\n'
                             'Стоит помнить, что от ежедневных занятий даже по 30 минут будет значительно больше пользы, чем '
                             'от одного занятия в неделю, но по 4 часа. Занятия должны быть регулярными, но не должны изматывать.\n\n'
                             '<b>2) Начните с разминки</b>\n'
                             'Прежде, чем начать играть нужно прогреть руки при помощи упражнений на хроматическую гамму. '
                             'Лучше не начинать разминаться с уже выученных Вами риффов, т.к. в этом случае Вы тратите тот ресурс сил, '
                             'который могли потратить на изучение нового материала (хоть и немного, но всё же). 5-10-тиминутной разминки '
                             'будет вполне достаточно.\n\n'
                             '<b>3) Сначала новое, потом старое</b>\n'
                             'В самом начале занятия у Вас, как правило, больше сил, чем в конце. Поэтому лучше начинайте либо с нового '
                             'материала, либо с того, который у вас не получался на прошлом занятии. Повторять уже выученные риффы лучше '
                             'в конце занятия.\n\n'
                             '<b>4) Изучайте соло, ритм и импровизацию</b>\n'
                             'Это поможет Вам стать всесторонне развитым музыкантом.\n\n'
                             '<b>5) Занимайтесь под метроном</b>\n'
                             'Сначала Вы будете ненавидеть его, но рано или поздно он станет Вашим лучшим другом, который будет помогать '
                             'не сбиваться с темпа.\n\n'
                             '<b>6) Занимайтесь с преподавателем</b>\n'
                             'На первый порах это просто необходимо, чтобы те ошибки, которые Вы будете допускать, не стали систематическими.\n\n'
                             '<b>7) Занимайтесь по видеоурокам</b>\n'
                             'На сегодняшний день очень много качественных видеоуроков по гитаре можно найти на Youtube. От себя '
                             'порекомендую подписаться на <a href = "https://www.youtube.com/c/FirstFretLessons">канал</a> и вступить в гитарное '
                             'сообщество <a href = "https://1lad.ru/">"Первый лад"</a>. У ребят очень '
                             'качественные бесплатные видеоуроки на канале для музыкантов всех уровней, а также множество интересных '
                             'платных видеокурсов по различным стилям музыки: блюз, рок, фанк, джаз, рок-н-ролл и др.', parse_mode="html")
            send_keyboard_1(call, 'Чем еще могу помочь?')
        
        elif call.text == 'Немного о струнах':
            bot.send_message(call.chat.id, "<b>Бренд</b> \nВыбор струн - очень важный момент для каждого гитариста. "
                             "Есть множество различных брендов, каждый из которых хорош по-своему. "
                             "Если Вы уже опытный музыкант, то наверняка определились для себя, что "
                             "Вам больше подходит по стилю. Если же Вы только открываете для себя этот "
                             "увлекательный мир электрогитар, то рекомендую для начала попробовать "
                             "Ernie Ball, D'Addario, Elixir. Уверен, что каждые из них Вас зацепят чем-то "
                             "своим, а со временем Вы уже сможете понимать, какого звука хотите добиться, "
                             "и какие струны Вам будут по душе. \n \n"
                             "<b>Калибр</b> \nТакже стоит обратить внимание на толщину и "
                             "силу натяжения струн. Если Вы планируете часто делать бенды, то рекомендую "
                             "рассмотреть струны от 8 до 10 калибра, так как тонкие струны тянутся лучше, "
                             "а также на них легче играть. Если же Вам по душе более тяжелая музыка, и "
                             "пониженные строи, то рекомендую рассмотреть струны потолще. От 12 калибра "
                             "и выше. Толстые струны имеют более богатый тембр, а также лучше держат строй, "
                             "но играть на них тяжелее, а также делать бенды на них не всегда комфортно. \n \n"
                             "<b>Натяжение</b> \nЧто касается силы натяжения, то здесь всё просто. Легкая натяжка даст возможность "
                             "извлекать звук проще и не уставать дольше, а более хардовая обеспечит более громкий звук.\n \n"
                             "Более подробно с этой темой можете ознакомиться по ссылке:\n" 
                             "https://www.youtube.com/watch?v=x1N3JhHi5Dw", parse_mode="html")
            send_keyboard_1(call, 'Чем еще могу помочь?')
        
        elif call.text == 'Известные музыканты':       
            bot.send_message(call.chat.id, 'А я сейчас Вам покажу любимые гитары некоторых известных музыкантов ;-) \n \n'
                             '<b>Jimi Hendrix</b> - <a href = "https://www.youtube.com/watch?v=_-7toYWFEyk"><i>Fender Stratocaster</i></a> \n \n'
                             '<b>Kurt Cobain</b> - <a href = "https://samesound.ru/inst/114959-fender-mustang-kurt-cobain"><i>Fender Mustang</i></a> \n \n'
                             '<b>Saul Hudson (Slash)</b> - <a href = "https://gibsonshop.ru/blog/obzory-poleznaja-informatsija/52-gitary-slesha-1997-2013.html"><i>Gibson Les Paul</i></a> \n \n'
                             '<b>Angus Young</b> - <a href = "https://www.youtube.com/watch?v=rRlTZ27q08w"><i>Gibson SG 1968</i></a> \n \n'
                             '<b>BB King</b> - <a href = "https://ru.wikipedia.org/wiki/%D0%9B%D1%8E%D1%81%D0%B8%D0%BB%D1%8C_(%D0%B3%D0%B8%D1%82%D0%B0%D1%80%D0%B0)"><i>Люсиль</i></a> \n \n'
                             '<b>Jimmy Page</b> - <a href = "https://www.muztorg.ru/news/dzhimmi-pejdzh-instrumentarij-volshebnika"><i>Fender Telecaster</i></a> \n \n'
                             '<b>Keith Richards</b> - <a href = "https://www.muztorg.ru/news/9945"><i>Fender Telecaster</i></a> \n \n'
                             '<b>Stephen (Stevie) Ray Vaughan</b> - <a href = "http://worldelectricguitar.ru/articles/srv_guitars.php"><i>Fender Stratocaster</i></a> \n \n'
                             '<b>Chuck Berry</b> - <a href = "https://www.gibson.com/en-US/Electric-Guitar/CUSRN3778/Vintage-Natural"><i>Gibson ES-355</i></a> \n \n'
                             '<b>James Hetfield</b> - <a href = "https://www.espguitars.ru/old/esp-guitars/signature/hetfield/esp-truckster-apg-james-hetfield.html"><i>ESP Truckster</i></a> \n \n'
                             '<b>Mark Nopfler</b> - <a href = "https://www.muztorg.ru/news/mark-nopfler-sekrety-zvuka"><i>Fender Stratocaster</i></a> \n \n'
                             '<b>Eric Clapton</b> - <a href = "https://www.muztorg.ru/news/erik-klepton-bog-igraet-na-gitare"><i>Fender Stratocaster</i></a>', parse_mode="html")
            send_keyboard_1(call, 'Чем еще могу помочь?')
    
        elif call.text =='Что делать после покупки?':
            bot.send_message(call.chat.id,'После приобретения новой гитары её необходимо отстроить под Вас: \n \n' 
                             '<b>1) Высота струн</b> \nНа новых инструментах высота струн отстраивается при помощи регулировки '
                             'высоты на бридже и прогиба грифа посредством анкерного болта. Высоту струн следует выбирать в зависимости '
                             'от силы атаки и выбранной толщины струн. Струны не должны задевать порожки грифа и звенеть '
                             'при игре на гитаре, и в то же время не должно возникать чрезмерное усилие для их зажатия.\n \n'
                             '<b>2) Новые струны</b> \nНа инструментах, только выпущенных с завода, обычно по умолчанию '
                             'установлены самые бюджетные струны, которые также за время пребывания в магазине '
                             'могли быть подвержены коррозии, т.к. находятся в постоянном контакте с воздухом. '
                             'Струны такого качества не смогут раскрыть звук Вашего нового инструмента во всей его красе, '
                             'поэтому рекомендую их заменить на новые, и Вы сразу же ощутите заметную разницу. '
                             'При замене заводских струн на более толстые, возможно потребуется немного расширить пазы верхнего '
                             'порожка гитары. \n \n'
                             '<b>3) Очистка грифа</b> \nПри замене струн рекомендую почистить накладку грифа лимонным маслом, '
                             'т.к. за время пребывания гитары в магазине на накладке грифа могла скопиться грязь, которая '
                             'будет в дальнейшем собираться струнами при их зажатии, что сильно сократит их срок службы.\n \n'
                             '<b>4) Отстройка мензуры</b> \nМензура - простыми словами это длина струны от верхнего порожка до бриджа. '
                             'От этого параметра зависит то, насколько чистые ноты будет брать ваша гитара, и насколько хорошо '
                             'она будет строить. Отсраивается мензура при помощи регулировки позиционнирующих винтов на бридже.\n \n'
                             '<b>ВАЖНО!</b>\nВы можете попытаться выполнить все перечисленные операции самостоятельно, но будет намного '
                             'лучше, если всё это сделает опытный гитарный мастер. Тем не менее, если Вам интересно, как выполняется '
                             'отстройка нового инструмента, можете посмотреть об этом в видео:\n'
                             'https://www.youtube.com/watch?v=LXQd2CuZUKc&t=19s',parse_mode="html")
            send_keyboard_1(call, 'Чем еще могу помочь?')
        
        elif call.text =='Первая электрогитара':
            bot.send_message(call.chat.id,'Итак, Вы решили приобрести свою первую электрогитару. Рекомендую в первую очередь '
                             'обратить внимание на более универсальные и бюджетные варианты инструмента. Почему? '
                             'Всё дело в том, что на первых порах Вам нужно понять какой именно звук Вам больше импонирует. '
                             'Если ранее Вы не имели опыта игры на гитаре, то для начала Вам лучше потратить определенное время, '
                             'чтобы поиграть разную музыку и понять, какая Вам нравится больше, и только после этого обратить '
                             'внимание на более узкоспециализированные инструменты. Для таких целей очень хорошо подойдет серия '
                             'Fender Squier. Это достаточно универсальный инструмент с хорошим звучанием за свою цену. \n \n'
                             'На нем можно играть самые различные стили музыки. Если Вы чувствуете уверенность в том, что хотите '
                             'развиваться в более тяжелых направлениях, то рекомендую обратить внимание на такие бренды, как Ibanez, '
                             'Epiphone, Jackson, Gibson. У данных производителей есть, как бюджетные варианты, так и продукция премиум-класса. \n \n'
                             'Если Ваш бюджет сильно ограничен, то изучите инструменты Ashtone. Данный бренд специализируется на гитарах '
                             'эконом-класса с приемлемым звучанием. Главным достоинством гитар Ashtone является их цена. Нередко '
                             'опытные музыканты также приобредают гитары этого производителя в качестве "походного" инструмента.')
            send_keyboard_1(call, 'Чем еще могу помочь?')
    
        elif call.text.strip().lower() in {'fender', 'fend', 'fendr', 
                                           'фендер', 'фендр',
                                           'фэндэр', 'фэндр', 'фендэр',
                                           'фенд', 'фэнд'}:
            bot.send_message(call.chat.id, '<b>Fender</b> – это знаменитый, не нуждающийся в представлении американский производитель гитар. '
                             'Компания была образована в г. Фуллертон (Калифорния) в 1946 году Кларенсом Леонидасом («Лео») Фендером. '
                             'В 1951 году Fender выпустил свою знаменитую модель электрогитары Telecaster (изначально называвшуюся '
                             'Broadcaster) и бас-гитару Precision Bass. В 1954 году свет увидела еще одна знаменитая модель, '
                             'ставшая визитной карточкой компании – <a href = "https://www.youtube.com/watch?v=wl7ZOyKNtAg&t=901s"><i>Stratocaster</i></a>. '
                             'Очень скоро гитары Fender стали самыми широко '
                             'распространенными и популярными. Классические модели электрогитар и бас-гитар Fender (Теле, Страт, '
                             'а также Jaguar, Jazzmaster, Mustang, Precision Bass и Jazz Bass) производятся до сих пор и мало изменились '
                             'с 50-х и 60-х годов. \n \nЗвук гитар Fender стал визитной карточкой рока и высоко ценится множеством гитаристов '
                             'по всему миру. Список звезд мировой величины, использующих и рекламирующих инструменты Fender огромен: это '
                             'такие знаменитые музыканты как Ричи Блэкмор, Джимми Пейдж, Дэвид Гилмор, Эрик Клэптон, Пол МакКартни, '
                             'Боб Дилан, Кейт Ричардс, Брюс Спрингстин, Ингви Мальмстин, и многие другие. '
                             'Об отличиях "стратов" и "телеков" можете посмотреть <a href = "https://www.youtube.com/watch?v=BHL-Fes47ik">здесь</a>\n\n'
                             '<b>Зарубежный веб-сайт:</b>\nhttp://intl.fender.com \n\n<b>Русскоязычный веб-сайт:</b>\nhttp://www.fender.ru\n\n'
                             '<b>Электрогитары Fender в магазинах:</b>\n'
                             '<a href = "https://www.muztorg.ru/search/Fender?maincat=gitary&subcat=elektrogitary">Музторг</a>\n'
                             '<a href = "https://pop-music.ru/search/?q=%D0%AD%D0%BB%D0%B5%D0%BA%D1%82%D1%80%D0%BE%D0%B3%D0%B8%D1%82%D0%B0%D1%80%D0%B0+Fender">Pop Music</a>\n'
                             '<a href = "https://dynatone.ru/index.php?route=product/search&search=fender&category_id=77876">Dynatone</a>', parse_mode="html")
            send_keyboard_2(call, 'О каком бренде рассказать еще?')
        
        elif call.text.strip().lower() in {'gibson', 'gibs', 'gib', 'gibsn',
                                           'gipson', 'gips', 'gip', 'gipsn',
                                           'гибсон', 'гибсн', 'гибс',
                                           'гипсон', 'гипсн', 'гипс',
                                           'джибсон', 'джибсн', 'джибс',
                                           'джипсон', 'джипсн', 'джипс'}:
            bot.send_message(call.chat.id, 'Инструменты <a href = "https://www.youtube.com/watch?v=s30AnsapM9A"><b>Gibson</b></a> имеют '
                             'широкую известность в мире с начала 20 века. '
                             'Первые гитары Gibson буквально перевернули гитарный мир, подарив музыкантам новое звучание, '
                             'оригинальный стиль и качество. Даже в настоящее время, эти инструменты представляют собой эталон '
                             'звучания и классику дизайнерской и инженерной мысли. Имя Gibson представляет более чем столетнюю '
                             'традицию производства одних из лучших инструментов. Среди известных гитаристов, в разное время игравших '
                             'на инструментах Gibson, значатся такие музыканты, как Би Би Кинг и Эрик Клэптон, Джимми Пэйдж и '
                             'Ангус Янг, Мэрилин Мэнсон и Слэш, Зак Уайлд, Серж Танкян, Джеймс Хэтфилд и многие другие. '
                             'Компания смогла навечно вписать свое имя в историю музыки, создав самые популярные, ставшие классикой '
                             'электро- и бас-гитары. \n \n <b>Официальный сайт:</b>\nhttp://www.gibson.com/\n\n'
                             '<b>Электрогитары Gibson в магазинах:</b>\n'
                             '<a href = "https://www.muztorg.ru/search/gibson?maincat=gitary&subcat=elektrogitary">Музторг</a>\n'
                             '<a href = "https://pop-music.ru/search/?q=%D0%AD%D0%BB%D0%B5%D0%BA%D1%82%D1%80%D0%BE%D0%B3%D0%B8%D1%82%D0%B0%D1%80%D0%B0+gibson">Pop Music</a>\n'
                             '<a href = "https://dynatone.ru/index.php?route=product/search&search=gibson&category_id=77876">Dynatone</a>', parse_mode="html")
            send_keyboard_2(call, 'О каком бренде рассказать еще?')
        
        elif call.text.strip().lower() in {'epiphone', 'epiphon', 'epipho', 'epiph', 'epip',
                                           'epifone', 'epifon', 'epifo', 'epif',
                                           'epephone', 'epephon', 'epepho', 'epeph', 'epep',
                                           'epefone', 'epefon', 'epefo', 'epef',
                                           'эпефон', 'эпифон', 'епефон', 'епифон',
                                           'эпефо', 'эпифо', 'епефо'
                                           'эпеф', 'эпиф', 'епеф', 'епиф'}:
            bot.send_message(call.chat.id, '<b>Epiphone</b> - это знаменитый американский производитель гитар, базирующийся в г. '
                             'Нэшвиль (Теннесси). История компании началась в далеком 1873 году, когда ее основатель, грек '
                             'Anastasios Stathopoulos начал производить собственные музыкальные инструменты. В 1903 году '
                             'Анастасиос переехал в Соединенные Штаты и там продолжил создавать свои инструменты, в том числе '
                             'мандолины. Дело отца продолжил сын Epaminondas «Epi», который стал также делать банджо. Компания '
                             'получила название Epiphone Banjo Company в 1928 году, и тогда же были выпущены первые гитары Epiphone. '
                             'В середине прошлого века <a href = "https://www.youtube.com/watch?v=suxsAXQh9Ig">Epiphone</a> выпускали '
                             'великолепные электрогитары с полым корпусом и были основным '
                             'конкурентом Gibson до тех пор, пока Gibson не удалось приобрести компанию в 1957 году, когда Epiphone '
                             'испытывали финансовые трудности. Одна из самых известных моделей того времени – это модель Casino, на '
                             'которой играл Джон Леннон. Бренд Epiphone существует до сих пор, являясь подразделением Gibson. Гитары '
                             'Epiphone отличаются великолепным качеством, сохранили свой индивидуальный стиль и характерное звучание, '
                             'при этом они гораздо более бюджетны по сравнению с инструментами Gibson.\n\n'
                             '<b>Официальный сайт:</b>\nhttp://www.epiphone.com\n\n'
                             '<b>Электрогитары Epiphone в магазинах:</b>\n'
                             '<a href = "https://www.muztorg.ru/search/epiphone?maincat=gitary&subcat=elektrogitary">Музторг</a>\n'
                             '<a href = "https://pop-music.ru/search/?q=%D0%AD%D0%BB%D0%B5%D0%BA%D1%82%D1%80%D0%BE%D0%B3%D0%B8%D1%82%D0%B0%D1%80%D0%B0+epiphone">Pop Music</a>\n'
                             '<a href = "https://dynatone.ru/index.php?route=product/search&search=epiphone&category_id=77876">Dynatone</a>', parse_mode="html")
            send_keyboard_2(call, 'О каком бренде рассказать еще?')
        
        elif call.text.strip().lower() in {'ibanez', 'ibane', 'iban', 'iba',
                                           'ibanes', 'ebanez', 'ebane', 'eban', 'eba',
                                           'ebanes', 'ibonez', 'ibone', 'ibon', 'ibo',
                                           'ibones', 'ebonez', 'ebone', 'ebon', 'ebo',
                                           'ebones',
                                           'ибанез', 'ибанэз', 'ибанес', 'ибанэс',
                                           'ибонез', 'ибонэз', 'ибонес', 'ибонэс'}:
            bot.send_message(call.chat.id, 'Компания <a href = "https://www.youtube.com/watch?v=o8QiQWBc-ak"><b>Ibanez</b></a> '
                             'была создана в 1908 году в Японии. Этот производитель стал одним '
                             'из первых, кто, начав производство гитар в Японии, смог добиться большого успеха в США, Европе '
                             'и по всему миру. Первое производство акустических гитар Ibanez, было начато в 1929 году. '
                             'Современная эпоха электро-и бас-гитар Ibanez стартовала в 1957 году. Также эта компания стала '
                             'одной из первых, кто начал производство 7-струнных электрогитар  и бас-гитар с пятью струнами. '
                             'Сегодня многие музыканты знают, что топовые и недорогие инструменты этого производителя имеют '
                             'отличное качество. Эндорсерами этого бренда являются такие музыканты, как Стив Вай и Пол Гилберт, '
                             'Мик Томсон (Slipknot) и Питер Иверс (InFlames), Джо Сатриани, Дарон Малкаян (System of a Down) и '
                             'многие другие.\n\n'
                             '<b>Зарубежный веб-сайт</b>:\n http://www.ibanez.com/eu/ \n'
                             '<b>Русскоязычный веб-сайт</b>:\n http://www.ibanez.ru/\n\n'
                             '<b>Электрогитары Ibanez в магазинах:</b>\n'
                             '<a href = "https://www.muztorg.ru/search/ibanez?maincat=gitary&subcat=elektrogitary">Музторг</a>\n'
                             '<a href = "https://pop-music.ru/search/?q=%D0%AD%D0%BB%D0%B5%D0%BA%D1%82%D1%80%D0%BE%D0%B3%D0%B8%D1%82%D0%B0%D1%80%D0%B0+ibanez">Pop Music</a>\n'
                             '<a href = "https://dynatone.ru/index.php?route=product/search&search=ibanez&category_id=77876">Dynatone</a>', parse_mode="html")
            send_keyboard_2(call, 'О каком бренде рассказать еще?')
    
        elif call.text.strip().lower() in {'jackson', 'jeckson', 'jacksun', 'jecksun', 'jacksan', 'jecksen',
                                          'jack', 'jeck', 'jacks', 'jecks', 'jackso', 'jeckso', 'jacksu', 'jecksu',
                                          'jacksa', 'jecksa',
                                          'jakson', 'jekson', 'jaksun', 'jeksun', 'jaksan', 'jeksen',
                                          'jak', 'jek', 'jaks', 'jeks', 'jakso', 'jekso', 'jaksu', 'jeksu',
                                          'jaksa', 'jeksa',
                                          'jacson', 'jecson', 'jacsun', 'jecsun', 'jacsan', 'jecsen',
                                          'jac', 'jec', 'jacs', 'jecs', 'jacso', 'jecso', 'jacsu', 'jecsu',
                                          'jacsa', 'jecsa',
                                          'джексон', 'джексэн', 'джексен', 'джексан',
                                          'джэксон', 'джэксэн', 'джэксен', 'джэксан',
                                          'джэк', 'джек', 'джекс', 'джэкс',
                                          'жэксон', 'жексэн', 'жексен', 'жексан'}:
            bot.send_message(call.chat.id, 'Гитары <a href = "https://www.youtube.com/watch?v=o8QiQWBc-ak"><b>Jackson</b></a> '
                             'ассоциируются с гитарами для тяжёлого рока - '
                             'и это так. Выпустив свои первые инструменты в 1980 году, Jackson стали одними из известных '
                             'законодателей моды в создании электрогитар для тяжёлой рок-музыки. На сегодняшний день, благодаря '
                             'своему вкладу в развитие тяжёлого рока, гитары Jackson продолжают пользоваться популярностью и имеют '
                             'свой широкий круг приверженцев и почитателей. С 2001 года компания Jackson вошла в группу компаний '
                             'Fender Musical Instruments. Среди известных гитаристов, использовавших инструменты Jackson есть такие '
                             'известные имена, как Adrian Smith (IRON MAIDEN), David Ellefson (MEGADETH), MarkMorton (LAMB OF GOD), '
                             'Phil Collen (DEF LEPPARD), Randy Rhoads (OZZY OSBOURNE), ScottIan (ANTHRAX) и многие другие музыканты.\n\n'
                             '<b>Официальный сайт:</b> \nhttp://www.jacksonguitars.com/\n\n'
                             '<b>Электрогитары Jackson в магазинах:</b>\n'
                             '<a href = "https://www.muztorg.ru/search/jackson?maincat=gitary&subcat=elektrogitary">Музторг</a>\n'
                             '<a href = "https://pop-music.ru/search/?q=%D0%AD%D0%BB%D0%B5%D0%BA%D1%82%D1%80%D0%BE%D0%B3%D0%B8%D1%82%D0%B0%D1%80%D0%B0+jackson">Pop Music</a>\n'
                             '<a href = "https://dynatone.ru/index.php?route=product/search&search=jackson&category_id=77876">Dynatone</a>', parse_mode="html")
            send_keyboard_2(call, 'О каком бренде рассказать еще?')
        
        elif call.text.strip().lower() in {'yamaha', 'yamah', 'yama',
                                           'iamaha', 'iamah', 'iama',
                                           'emaha', 'emah', 'ema',
                                           'ямаха', 'ямах', 'яма',
                                           'емаха', 'емах', 'ема'}:
            bot.send_message(call.chat.id, 'Корпорация <a href = "https://www.youtube.com/watch?v=8vsMFGnZsN0"><b>Yamaha</b></a> '
                             'была основана в Японии в 1887 году и начала свою деятельность '
                             'с производства язычковых органов (фисгармоний). С тех пор Yamaha выросла в крупнейшего в мире '
                             'производителя большинства видов музыкальных инструментов и профессионального аудио-оборудования. '
                             'На счету компании целый ряд революционных новшеств, изменивших лицо мировой индустрии музыкальных '
                             'инструментов и оборудования, а также оказавших огромное влияние на целые поколения музыкантов всех '
                             'жанров и направлений.\n\n'
                             '<b>Зарубежный веб-сайт:</b> \nhttps://www.yamaha.com/ \n\n'
                             '<b>Русскоязычный веб-сайт:</b> \nhttp://ru.yamaha.com/\n\n'
                             '<b>Электрогитары Yamaha в магазинах:</b>\n'
                             '<a href = "https://www.muztorg.ru/search/yamaha?maincat=gitary&subcat=elektrogitary">Музторг</a>\n'
                             '<a href = "https://pop-music.ru/search/?q=%D0%AD%D0%BB%D0%B5%D0%BA%D1%82%D1%80%D0%BE%D0%B3%D0%B8%D1%82%D0%B0%D1%80%D0%B0+yamaha">Pop Music</a>\n'
                             '<a href = "https://dynatone.ru/index.php?route=product/search&search=yamaha&category_id=77876">Dynatone</a>', parse_mode="html")
            send_keyboard_2(call, 'О каком бренде рассказать еще?')
        
        elif call.text.strip().lower() in {'ashtone', 'ashton', 'eshtone', 'eshton',
                                           'ashto', 'eshto', 'asht', 'esht',
                                           'achtone', 'achton', 'echtone', 'echton',
                                           'achto', 'echto', 'acht', 'echt',
                                           'aschtone', 'aschton', 'eschtone', 'eschton',
                                           'aschto', 'eschto', 'ascht', 'escht',
                                           'эштон', 'аштон', 'эшто', 'ашто', 'эшт', 'ашт'}:
            bot.send_message(call.chat.id, 'Компания <b>ASHTONE</b> занимается производством бас-гитар, электроакустических и '
                             'электрогитар начального уровня, руководствуясь принципом: хорошая гитара не обязательно '
                             'должна стоить дорого. Обобщив лучший мировой опыт производства гитар, технологам ASHTONE '
                             'удалось создать очень хорошие и недорогие инструменты. При производстве бас- и электрогитар '
                             'ASHTONE используются лучшие породы древесины, надежная фурнитура и электроника. '
                             'Гитары <a href = "https://www.youtube.com/watch?v=UIawQdRgkUk"><b>ASHTONE</b></a> '
                             'хорошо зарекомендовали себя во всем мире. Они очень популярны среди начинающих музыкантов. '
                             'Многие профессионалы приобретают себе эти гитары для игры в "полевых условиях", когда не хочется '
                             'рисковать дорогими авторскими инструментами. Гитары ASHTONE производятся на одной из лучших фабрик '
                             'в Юго-Восточной Азии. Качество производства настолько высоко, что на этой фабрике размещают заказы '
                             'на производство гитар именитые компании, такие как WASHBURN, IBANEZ и многие другие.\n\n'
                             '<b>Электрогитары Ashtone в магазине:</b>\n'
                             '<a href = "https://pop-music.ru/search/?q=%D0%AD%D0%BB%D0%B5%D0%BA%D1%82%D1%80%D0%BE%D0%B3%D0%B8%D1%82%D0%B0%D1%80%D0%B0+ashtone">Pop Music</a>\n', 
                             parse_mode="html")
            send_keyboard_2(call, 'О каком бренде рассказать еще?')
    
        elif call.text == 'Порода дерева':
            bot.send_message(call.chat.id, 'На что влияет материал корпуса, грифа и накладки так в 2-х словах и не расскажешь. '
                             'Рекомендую ознакомиться вот с этой интересной <a href = "https://pop-music.ru/articles/derevo-dlya-gitary/#:~:text=%D0%A7%D0%B0%D1%89%D0%B5%20%D0%B2%D1%81%D0%B5%D0%B3%D0%BE%20%D0%B3%D1%80%D0%B8%D1%84%20%D0%B3%D0%B8%D1%82%D0%B0%D1%80%D1%8B%20%D0%B8%D0%B7%D0%B3%D0%BE%D1%82%D0%B0%D0%B2%D0%BB%D0%B8%D0%B2%D0%B0%D0%B5%D1%82%D1%81%D1%8F,%D0%BA%D0%BB%D0%B5%D0%BD%2C%20%D0%BF%D0%B0%D0%BB%D0%B8%D1%81%D0%B0%D0%BD%D0%B4%D1%80%2C%20%D1%87%D0%B5%D1%80%D0%BD%D0%BE%D0%B5%20%D0%B4%D0%B5%D1%80%D0%B5%D0%B2%D0%BE.">'
                             'статьей</a>, в ней всё очень подробно рассказано.', 
                              parse_mode="html")
            send_keyboard_3(call, 'О чем еще хотите узнать?')
        
        elif call.text == 'Тип корпуса':
            bot.send_message(call.chat.id, 'Есть одна интересная '
                             '<a href = "https://guitargrad.ru/elektrogitara/obshchaya-informatsiya/49-vidy-korpusov-elektrogitary">'
                             'статья</a>, в в которой коротко рассказана самая основная информация о типах корпусов электрогитар. \n\n'
                             'Если Вы любите джазовую музыку и хотите развиваться в этом направлении, то рекомендую обратить внимание '
                             'на <a href = "https://pop-music.ru/articles/vse-o-poluakusticheskikh-gitarakh/">полуакустические</a> или, '
                             'как иногда говорят, "полые" гитары.', parse_mode="html")
            send_keyboard_3(call, 'О чем еще хотите узнать?')
        
        elif call.text == 'Гриф':
            bot.send_message(call.chat.id, '<a href = "https://guitargrad.ru/elektrogitara/obshchaya-informatsiya/305-gitarnyj-grif">Здесь</a> Вы сможете найти '
                             'много интересной информации по конструкции грифа, способу его крепления к корпусу, а также на что это влияет.', parse_mode="html")
            send_keyboard_3(call, 'О чем еще хотите узнать?')
        
        elif call.text == 'Звукосниматели':
            bot.send_message(call.chat.id, 'Звукосниматели на электрогитаре - один из ключевых элементов, влюящих на её звук. '
                             'Прежде, чем выбрать инструмент, нужно обязательно ознакомиться с этой '
                             '<a href = "https://www.youtube.com/watch?v=fp_x31MjccM">темой </a> детальнее.'
                             'Некоторые музыканты предпочитают регулировать высоту звукоснимателей, для достижения большей яркости или '
                             'наоборот глухости звучания, а также чтобы управлять звуком атаки по струнам. Если Вам интересен этот момент, '
                             'то рекомендую озанчкомиться с этим <a href = "https://www.youtube.com/watch?v=iN501eRXPA8">видео</a>.', parse_mode="html")
            send_keyboard_3(call, 'О чем еще хотите узнать?')
    
        elif call.text == 'Настройка тембра':
            bot.send_message(call.chat.id, 'Помимо крутящейся ручки громкости гитары на ней можно также заметить '
                             'ещё одну, две или даже три дополнительные. Они называются ручками потенциометров и '
                             'используются для того, чтобы отрегулировать громкость '
                             'конкретных звукоснимателей (бриджевый, нэковый или средний) и выстроить тем самым необходимый тембр звука '
                             'инструмента. Также этих целей можно добиться путем регулировки высоты звукоснимателей. Поделюсь с Вами '
                             '<a href = "https://www.youtube.com/watch?v=Kv290YIKYiY">примером</a> '
                             'о том, как можно добиться "метольного" звука на Fender Stratocaster с 3 синглами.\n\n'
                             'Что касается <a href = "https://www.youtube.com/watch?v=kdW3IHkr4aI">Switch</a>. Это небольшой небольшой рычажок, который переключает возможные варианты одновременно '
                             'работающих звукоснимателей, что также сильно влияет на тембр инструмента.\n\n '
                             'Рекомендую обратить внимание на расположение всех этих элементов. С одной стороны, они должны находиться '
                             'всегда под рукой, чтобы можно было варьировать звук во время исполнения чего-либо, с другой '
                             'же они могут мешать Вашей правой руке во время игры. Однозначного мнения по правильному расположению '
                             'данных элементов нет. Кому-то нравится, что ручка громкости на гитарах Stratocaster располагается '
                             'рядом со струнами, т.к. это дает возможность задействовать его мизинцем правой руки во время игры, '
                             'а кто-то наоборот, считает это неудобным, т.к. можно случайно его задеть и изменить громкость гитары. '
                             'На инструментах Les Paul часто switch располагается в верхней части корпуса, что практически исключает '
                             'возможность его переключить случайно за исключением ситуации, когда Вы играете тэппингом. '
                             'В этом случае многие по началу часто задевают этот маленький рычажок. Однако, ко всему можно со временем '
                             'и практикой привыкнуть и приноровиться.', parse_mode="html")
            send_keyboard_3(call, 'О чем еще хотите узнать?')
    
        elif call.text == 'Рычаг тремоло':
            bot.send_message(call.chat.id, 'Рычаг тремоло является очень интересным, но абсолютно необязательным элементом электрогитары. '
                             'Об этом можно рассказывать достаточно долго, но лучше это 1 раз '
                             '<a href = "https://www.youtube.com/watch?v=cUVnX3kqQt8">увидеть</a> и решить для себя, '
                             'будете ли Вы им пользоваться. ', parse_mode="html")
            send_keyboard_3(call, 'О чем еще хотите узнать?')

        elif call.text == 'Колки':
            bot.send_message(call.chat.id, '<a href = "https://pop-music.ru/articles/vsye-o-gitarnykh-kolkakh/">Колки </a>'
                             'располагаются на голове грифа гитары, и служат для крепления струн. Жесткость их механизма '
                             'определяет то, насколько точно Ваш инструмент будет держать строй по нотам. На этот элемент следует '
                             'обратить внимание, если Вы планируете играть бэнды ("подтяжки струн") '
                             'или использовать рычаг тремоло. Частое применение таких приемов '
                             'игры, как правило, приводит к более частой необходимости настройки инструмента по тюнеру '
                             '(или "наслух" :-) ). Более жестие колки, позволят гитаре лучше держать строй и реже "слетать" с нот. '
                             'Также материал самих колков может оказывать влияние на звук гитары. \n\n'
                             'При осмотре гитары в магазине, обратите внимание на то, чтобы колки не издавали звон.', parse_mode="html")
            send_keyboard_3(call, 'О чем еще хотите узнать?')        
    
        elif call.text == 'Бридж':
            bot.send_message(call.chat.id, '<a href = "https://uchenikspb.ru/kbase/bridzh-na-gitare/">Бридж</a> располагается в '
                             'нижней части гитары и служит для крепления струн. При помощи бриджа отстраивается мензура '
                             '(длина струны) гитары. Точность отстройки рекомендую проверять по тюнеру. Открытая струна '
                             'и нота зажатая на 12 ладу должны совпадать, как можно точнее. Также на бридже регулируется высота струн. \n\n'
                             'Музыканты, играющие давно и хорошо знающие особенности своей игры, могут проводить все операции с бриджем '
                             'самостоятельно. Тем не менее рекомендую Вам всё-таки обратиться к опытному мастеру при покупке инструмента, '
                             'чтобы он смог "подогнать" его именно под Вас.', parse_mode="html")  
            send_keyboard_3(call, 'О чем еще хотите узнать?')                           
    
        elif call.text == 'Найти гитару в магазине':
            bot.send_message(call.chat.id, 'Я выполняю поиск музыкальных инструментов в магазинах "Музторг", "Pop-Music" и "Dynatone" '
                             'и присылаю ссылки на то, что сумел найти.')
            send_keyboard_4(call, 'Напишите мне название гитары или его часть.')
    
        elif call.text == 'Пока всё!':
            bot.send_message(call.chat.id, 'Рад был помочь! Чтобы пообщаться снова нажмите или напишите "/start"')
            
        elif call.text == 'История поиска':
            show_history(call)
            send_keyboard_4(call, 'Чем еще могу помочь?')
        
        elif call.text == 'Удалить экземпляр поиска':
            bot.send_message(call.chat.id,
                             'Пришлите мне номер поиска, который Вы бы хотели удалить. '
                             'Его можно посмотреть в истории поиска. Если Вы хотите удалить несколько '
                             'поисков, перечислите их номера через <b>ПРОБЕЛ</b> и отправьте мне ;-)', parse_mode="html" )
            bot.register_next_step_handler(call, delete_search)
            send_keyboard_4(call, 'Чем еще могу помочь?')
            
        elif call.text == 'Очистить истроию':
            delete_all_searches(call)
            send_keyboard_4(call, 'Чем еще могу помочь?')
        
        else:
            if call.text.strip().isdigit():
                send_keyboard_4(call, 'Если вы хотите удалить какой-либо поиск из истории, '
                                'нажмите кнопку "Удалить экземпляр поиска", затем отправьте мне его номер')
            else:
                search(call)
                send_keyboard_4(call, 'Чем еще могу помочь?')
            
    except:
        bot.send_message(call.chat.id, 'Что-то пошло не так :-( Давайте, попробуем ещё раз')
        send_keyboard_1(call, 'Чем я могу помочь?')
    
                  

In [14]:
bot.polling(none_stop=True)

2022-10-04 17:30:50,641 (__init__.py:660 MainThread) ERROR - TeleBot: "Threaded polling exception: A request to the Telegram API was unsuccessful. Error code: 403. Description: Forbidden: bot was blocked by the user"
2022-10-04 17:30:50,642 (__init__.py:662 MainThread) ERROR - TeleBot: "Exception traceback:
Traceback (most recent call last):
  File "C:\Users\79629\anaconda3\lib\site-packages\telebot\__init__.py", line 651, in __threaded_polling
    self.worker_pool.raise_exceptions()
  File "C:\Users\79629\anaconda3\lib\site-packages\telebot\util.py", line 147, in raise_exceptions
    raise self.exception_info
  File "C:\Users\79629\anaconda3\lib\site-packages\telebot\util.py", line 93, in run
    task(*args, **kwargs)
  File "C:\Users\79629\AppData\Local\Temp\ipykernel_24268\2207724948.py", line 17, in send_keyboard_1
    msg = bot.send_message(message.from_user.id,
  File "C:\Users\79629\anaconda3\lib\site-packages\telebot\__init__.py", line 1006, in send_message
    apihelper.send_m